In [61]:
import numbers
import math
import operator

class Expr():
    def __str__(self):
        return self.display()
    
    def __mul__(self, other):
        return Prod(self, other)

    def __rmul__(self, other):
        return Prod(other, self)    
    
    def __add__(self, other):
        return Sum(self, other)

    def __radd__(self, other):
        return Sum(other, self)
    
    
    def __truediv__(self, other):
        return Div(self, other)

    def __rtruediv__(self, other):
        return Div(other, self)
    
    def evaluate(self, **kwd):
        raise Exception('Cannot evaluate {self.__class__.__name__}'.format(self=self))
        
    
    
    
class Node(Expr):
    pass

    
class Leave(Expr):
    pass

class Symbol(Leave):
    def __init__(self, name):
        assert isinstance(name, str), 'name should be a str'
        self.name = name
        
    def display(self):
        return self.name        
    
    def __repr__(self):
        return 'Symbol({})'.format(self.name)

    def evaluate(self, **kwd):
        try:
            return kwd[self.name]
        except KeyError:
            raise Exception('Cannot evaluate variable {self.name}'.format(self=self))
    
class Number(Leave):
    def __init__(self, val):
        self.val = val
        
    def display(self):
        return str(self.val)
        
    def __repr__(self):
        return 'Number({})'.format(self.val)
    
    def evaluate(self, **kwd):
        return self.val
    
class Function(Node):
    pass

class MathFunction(Node):
    def __init__(self, arg):
        if isinstance(arg, numbers.Number):
            arg = Number(arg)
        assert isinstance(arg, Expr)
        self.arg = arg
        
    def display(self):
        return '{}({})'.format(self.function_name, 
                               self.arg.display())
    
    def __repr__(self):
        arg = repr(self.arg)
        return '{self.__class__.__name__}({arg})'.format(self=self, arg=arg)
    
    def evaluate(self, **kwd):
        return getattr(math, self.function_name)(self.arg.evaluate(**kwd))
    

class Sin(MathFunction):
    function_name = 'sin'
    derivee = lamdba x:1 + Tan(x)**2
    

class BinaryOperator(Function):
    def __init__(self, arg1, arg2):
        if isinstance(arg1, numbers.Number):
            arg1 = Number(arg1)
        if isinstance(arg2, numbers.Number):
            arg2 = Number(arg2)
        self.arg1 = arg1
        self.arg2 = arg2
        
    def evaluate(self, **kwd):
        left = self.arg1.evaluate(**kwd)
        right = self.arg2.evaluate(**kwd)
        return self.operator_function(left, right)
        
    
    def display(self):
        left = self.arg1.display()
        right = self.arg2.display()
        return "{left} {self.operator_name} {right}".format(left=left,
                                                            right=right,
                                                            self=self)
    
    def __repr__(self):
        left = repr(self.arg1)
        right = repr(self.arg2)
        return "{self.__class__.__name__}({left}, {right})".format(left=left,
                                                            right=right,
                                                            self=self)
        

class Sum(BinaryOperator):
    operator_name = '+'
    operator_function = operator.add
        
class Prod(BinaryOperator):
    operator_name = '*'
    operator_function = operator.mul


class Div(BinaryOperator):
    operator_name = '/'
    operator_function = operator.truediv

    
class Sub(BinaryOperator):
    operator_name = '-'
    operator_function = operator.sub
    

# Idem pour Sub, Div, Prod, Pow
class UnitaryOperator(Function):
    pass

class Neg(UnitaryOperator):
    pass

In [62]:
x = Symbol('x')
y = Symbol('y')
expr = Sum(x, Sin(Prod(x, y)))
expr

Sum(Symbol(x), Sin(Prod(Symbol(x), Symbol(y))))

In [63]:
(y+Sin(x)/2).evaluate(x=2, y=1)

1.454648713412841

In [57]:
print((y+Sin(x)/2))

y + sin(x) / 2


In [23]:
isinstance(x, Expr)

True

In [51]:
operator.truediv(1, 2)

0.5

In [52]:
operator.floordiv(1, 2)

0